Importing all the necessary dependencie

In [1]:
# import libraries
import tensorflow as tf
import pandas as pd
pd.options.display.max_columns = 50
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
plt.style.use('bmh')
# packages for data cleaning function
import re
import string

2022-06-25 22:22:47.036109: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Importing the dataset

In [5]:
df = pd.read_csv('/home/rumbleftw/Documents/Codes/global-censorship/datasets/text/clean_data.csv')
df.head


<bound method NDFrame.head of         is_offensive                                               text
0                  0  Then go to the village pump and suggest they c...
1                  1  ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...
2                  1     Dis hoe wasnt dis violent on Lottery Ticket 😂😂
3                  0  It is better for Atabay not helping the banned...
4                  0  "is in CamelCase.  ""SiCKO"" is not CamelCase,...
...              ...                                                ...
184349             0                   Template:uw-vandalism2 >  | Talk
184350             1   Regrets are for pussies. Shit happens, deal w...
184351             0  Could this possibly be the origin of popular g...
184352             0  "Your article submission has been declined, an...
184353             0  .\nEditors can not move articles except inside...

[184354 rows x 2 columns]>

Creating new column label for hate speech binary


In [7]:
df['is_offensive'].unique()

array([0, 1])

In [24]:
# function to clean all data
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    # make text lowercase
    text = str(text)
    text = text.lower()
    # removing text within brackets
    text = re.sub('\[.*?\]', '', text)
    # removing text within parentheses
    text = re.sub('\(.*?\)', '', text)
    # removing numbers
    text = re.sub('\w*\d\w*', '', text)
    # if there's more than 1 whitespace, then make it just 1
    text = re.sub('\s+', ' ', text)
    # if there's a new line, then make it a whitespace
    text = re.sub('\n', ' ', text)
    # removing any quotes
    text = re.sub('\"+', '', text)
    # removing &amp;
    text = re.sub('(\&amp\;)', '', text)
    # removing any usernames
    text = re.sub('(@[^\s]+)', '', text)
    # removing any hashtags
    text = re.sub('(#[^\s]+)', '', text)
    # remove `rt` for retweet
    text = re.sub('(rt)', '', text)
    # string.punctuation is a string of all punctuation marks
    # so this gets rid of all punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # getting rid of `httptco`
    text = re.sub('(httptco)', '', text)
    #########################################
    regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
                           "]+", flags = re.UNICODE)

    text = regex_pattern.sub(r'', text)

    return text

round1 = lambda x: clean_text_round1(x)

In [25]:
# appling data cleaning function to `tweet` column
df['text'] = df['text'].apply(round1)

In [26]:
# checking that it worked
df.head()

,is_offensive,text
0,0,then go to the village pump and suggest they c...
1,1,anti greek nationalis wikipedia hi alexikoua y...
2,1,dis hoe wasnt dis violent on lottery ticket
3,0,it is better for atabay not helping the banned...
4,0,is in camelcase sicko is not camelcase so the ...


In [29]:
df.tail()

,is_offensive,text
184349,0,templateuw talk
184350,1,regrets are for pussies shit happens deal wit...
184351,0,could this possibly be the origin of popular g...
184352,0,your aicle submission has been declined and wi...
184353,0,editors can not move aicles except inside use...


In [30]:
df.to_csv('/home/rumbleftw/Documents/Codes/global-censorship/datasets/text/finalData.csv')

In [2]:
df = pd.read_csv('/home/rumbleftw/Documents/Codes/global-censorship/datasets/text/finalData.csv')

In [3]:
df['text'] = df['text'].values.astype('str')

**Tokenizing the word vocab**

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<shobdobhandarErBahire>'
tokenizer.fit_on_texts(df['text'])
vocab = tokenizer.word_index
vocabSize = len(vocab)
vocabSize, vocab


(217799,
 {'<shobdobhandarErBahire>': 1,
  'the': 2,
  'to': 3,
  'and': 4,
  'of': 5,
  'a': 6,
  'you': 7,
  'i': 8,
  'is': 9,
  'that': 10,
  'in': 11,
  'it': 12,
  'for': 13,
  'this': 14,
  'not': 15,
  'on': 16,
  'be': 17,
  'as': 18,
  'have': 19,
  'are': 20,
  'your': 21,
  'with': 22,
  'if': 23,
  'aicle': 24,
  'was': 25,
  'but': 26,
  'or': 27,
  'my': 28,
  'page': 29,
  'an': 30,
  'from': 31,
  'do': 32,
  'by': 33,
  'at': 34,
  'me': 35,
  'so': 36,
  'about': 37,
  'wikipedia': 38,
  'can': 39,
  'what': 40,
  'all': 41,
  'there': 42,
  'will': 43,
  'has': 44,
  'like': 45,
  'please': 46,
  'no': 47,
  'would': 48,
  'its': 49,
  'just': 50,
  'they': 51,
  'one': 52,
  'dont': 53,
  'he': 54,
  'been': 55,
  'any': 56,
  'talk': 57,
  'should': 58,
  'we': 59,
  'which': 60,
  'more': 61,
  'some': 62,
  'other': 63,
  'who': 64,
  'here': 65,
  'im': 66,
  'see': 67,
  'his': 68,
  'think': 69,
  'also': 70,
  'know': 71,
  'because': 72,
  'how': 73,
  'am'

In [69]:
vocab['bloody']

2413

**Sequencing the whole dataset and padding to uniform shape**

In [5]:
textData = tokenizer.texts_to_sequences(df['text'])
paddedTextData = tf.keras.preprocessing.sequence.pad_sequences(textData, padding='pre')
paddedTextData.shape, paddedTextData

((184354, 1403),
 array([[    0,     0,     0, ...,    17,   669,    75],
        [    0,     0,     0, ...,    21,  7480,  1142],
        [    0,     0,     0, ...,    16, 16598,  7024],
        ...,
        [    0,     0,     0, ...,   548,   794,  9428],
        [    0,     0,     0, ...,     3,    38,    57],
        [    0,     0,     0, ...,     5, 17461,    82]], dtype=int32))

**Splitting into training and testing datasets**

In [29]:
trainLength = 10000


xTrain = paddedTextData[:trainLength]
yTrain =df['is_offensive'].to_numpy()[:trainLength]

xTest = paddedTextData[trainLength:12000]
yTest = df['is_offensive'].to_numpy()[trainLength:12000]

In [33]:
xTest[0], yTest[0]

(array([  0,   0,   0, ...,  16, 400,  75], dtype=int32), False)

In [34]:
df['is_offensive'][0]

False

**Defining the model**

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocabSize, 64, input_length=1403))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(38, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-06-25 22:24:41.646921: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-25 22:24:41.667121: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-25 22:24:41.667688: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-25 22:24:41.668317: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LUFFY): /proc/driver/nvidia/version does not exist
2022-06-25 22:24:41.671943: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow 

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1403, 64)          13939136  
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 38)                2470      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 39        
Total params: 13,941,645
Trainable params: 13,941,645
Non-trainable params: 0
_________________________________________________________________


**Traning Model**

In [35]:
model.fit(xTrain, yTrain, epochs=1)

313/313 [==============================] - 135s 433ms/step - loss: 0.4805 - accuracy: 0.8012


In [38]:
model.evaluate(xTest, yTest)

63/63 [==============================] - 1s 8ms/step - loss: 0.4599 - accuracy: 0.8080


[0.4598709046840668, 0.8080000281333923]

In [46]:
inpt =["i need molly water and a chubby asian bitch", "hello"]

seq = tokenizer.texts_to_sequences(inpt)
print(seq)

text = tf.keras.preprocessing.sequence.pad_sequences(seq, padding='pre', maxlen=1403)

model.predict(text)

[[8, 114, 11138, 1342, 4, 6, 24948, 2134, 126], [284]]


array([[0.21021113],
       [0.21095604]], dtype=float32)